## Load Data Set

In [1]:
import zipfile
import os
import numpy as np
import pathlib
import pandas as pd
from math import ceil
import tensorflow as tf
import numpy as np
import IPython.display as display
import keras
from keras import backend as K
from matplotlib import pyplot as plt
from keras.utils import to_categorical


print(tf.__version__)

root_path = './'
dataset_path = os.path.join(root_path, 'tf_dataset')

models_path = os.path.join(root_path, 'saved_models')
if not os.path.exists(models_path):
  os.mkdir(models_path)

Using TensorFlow backend.


1.13.1


In [2]:
BATCH_SIZE = 16


image_feature_description = {
    'enc': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
    'img': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True)
}


def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, image_feature_description)


raw_train_dataset = tf.data.TFRecordDataset(os.path.join(dataset_path,'train.tfrecords'))
parsed_train_dataset = raw_train_dataset.map(_parse_image_function)

raw_val_dataset = tf.data.TFRecordDataset(os.path.join(dataset_path, 'val.tfrecords'))
parsed_val_dataset = raw_val_dataset.map(_parse_image_function)

raw_test_dataset = tf.data.TFRecordDataset(os.path.join(dataset_path, 'test.tfrecords'))
parsed_test_dataset = raw_test_dataset.map(_parse_image_function)


parsed_train_dataset = parsed_train_dataset.repeat()
parsed_train_dataset = parsed_train_dataset.shuffle(1000)
parsed_train_dataset = parsed_train_dataset.batch(BATCH_SIZE)
dataset_iterator = parsed_train_dataset.make_one_shot_iterator()

variable_dataset = dataset_iterator.get_next()

## Wasserstein GAN

In [3]:
def d_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

## Model Design

In [4]:
enc_len = 128
age_len = 8
img_shape = (32, 32, 3)
width, height, depth = (32, 32, 3)
img_len = np.prod(img_shape)
latent_dim = enc_len + age_len + img_len
noise_len = 100
input_dim = enc_len + age_len + noise_len
cond_len = enc_len + age_len


def build_discriminator():
    conv = keras.Sequential([
        # conv block 1
        keras.layers.Conv2D(
            filters=16,
            kernel_size=(3, 3),
            strides=2,
            input_shape=img_shape
        ),
        keras.layers.Activation(tf.nn.leaky_relu),

        # conv block 2
        keras.layers.Conv2D(
            filters=32,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 3
        keras.layers.Conv2D(
            filters=3,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
    ])
    
    conv.summary()
    
    model = keras.Sequential([
        # dense 1
        keras.layers.Dense(128, input_shape=(27+128+8,)),
        keras.layers.Activation(tf.nn.relu),
        
        # output
        keras.layers.Dense(1),
        keras.layers.Activation(tf.nn.sigmoid),
    ])
    
    clf = keras.Sequential([
        # dense 1
        keras.layers.Dense(128, input_shape=(27+128+8,)),
        keras.layers.Activation(tf.nn.relu),
        
        # output
        keras.layers.Dense(age_len),
        keras.layers.Activation(tf.nn.softmax),
    ])
    
    model.summary()
    
    # condition
    c1 = keras.layers.Input(shape=(enc_len,))
    c2 = keras.layers.Input(shape=(age_len,))
    
    # image
    z = keras.layers.Input(shape=img_shape)
    
    # convolution
    zout = conv(z)
    
    # flatten image
    z_flat = keras.layers.Flatten()(zout)
    
    # concatenation
    inputs = keras.layers.concatenate([c1, c2, z_flat])
    
    # real or fake
    outputs = model(inputs)
    
    # age label
    classes = clf(inputs)
    
    
    return keras.models.Model([c1, c2, z], [outputs, classes])


def build_generator():
    
    conv = keras.Sequential([
        # transpose conv block 1
        keras.layers.Conv2DTranspose(
            filters=16,
            kernel_size=(3, 3),
            strides=1,
            input_shape=(11, 11, 1)
        ),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 2
        keras.layers.Conv2DTranspose(
            filters=32,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 3
        keras.layers.Conv2DTranspose(
            filters=32,
            kernel_size=(2, 2),
            strides=1
        ),
        keras.layers.Activation(tf.nn.relu),

        # transpose conv block 4
        keras.layers.Conv2DTranspose(
            filters=3,
            kernel_size=(5, 5),
            strides=1
        ),
        
        # output
        keras.layers.Activation(tf.nn.tanh)
    ])
    
    conv.summary()
    
    model = keras.Sequential([
        # dense 1
        keras.layers.Dense(121, input_shape=(input_dim,)),
        keras.layers.Activation(tf.nn.relu),
        
        # reshape 1d to 3d
        keras.layers.Reshape((11, 11, 1))
    ])
    
    model.summary()
    
    # condition
    c1 = keras.layers.Input(shape=(enc_len,))
    c2 = keras.layers.Input(shape=(age_len,))
    
    # noise
    x = keras.layers.Input(shape=(noise_len,))

    # concatenation
    inputs = keras.layers.concatenate([c1, c2, x])
    
    # flat dense output
    out_1 = model(inputs)
    
    # transpose conv output
    outputs = conv(out_1)
    
    return keras.models.Model([c1, c2, x], outputs)


discriminator = build_discriminator()
generator = build_generator()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 15, 15, 16)        448       
_________________________________________________________________
activation_1 (Activation)    (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
activation_2 (Activation)    (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 3)           867       
_________________________________________________________________
activation_3 (Activation)    (None, 3, 3, 3)           0         
Total params: 5,955
Trainable params: 5,955
Non-trainable params: 0
__

In [5]:
generator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 8)            0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 236)          0           input_4[0][0]                    
                                                                 input_5[0][0]                    
          

In [6]:
discriminator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 3, 3, 3)      5955        input_3[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 8)            0                                            
__________________________________________________________________________________________________
flatten_1 

## Generative Adversarial Network

In [7]:
GLR = 0.002  # generator
DLR = 0.002  # discriminator


discriminator.compile(
    optimizer=keras.optimizers.Adam(DLR, 0.5),
    loss=[d_loss, keras.losses.categorical_crossentropy],
    metrics=['accuracy']
)


# condition
c1 = keras.layers.Input(shape=(enc_len,))
c2 = keras.layers.Input(shape=(age_len,))

# noise
x = keras.layers.Input(shape=(noise_len,))

# freeze discriminator
discriminator.trainable = False

# output
z = generator([c1, c2, x])
out, lbl = discriminator([c1, c2, z])

# GAN
gan = keras.models.Model(inputs=[c1, c2, x], outputs=[out, lbl])

gan.compile(
    optimizer=keras.optimizers.Adam(GLR , 0.5),
    loss=[d_loss, keras.losses.categorical_crossentropy],
    metrics=['accuracy'])

In [8]:
gan.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 8)            0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, 32, 32, 3)    40008       input_7[0][0]                    
                                                                 input_8[0][0]                    
          

## Visualization Method

In [9]:
# from google.colab import drive
import os


# drive.mount('/content/gdrive', force_remount=True)

root_path = './'
tgt_pth = os.path.join(root_path, 'visualize_age-cgan-v2')

if not os.path.exists(tgt_pth):
  os.mkdir(tgt_pth)

In [10]:
def visualizeGAN(e, z_real, z_fake):

    fig, axes = plt.subplots(8, 4, figsize=(40, 36))

    r_real = 0
    r_fake = 0
    for row, axe in enumerate(axes):
        for col, cell in enumerate(axe):
            if row % 2 == 0:
                cell.imshow(z_real[r_real * 4 + col])
            else:
                cell.imshow((z_fake[r_fake * 4 + col] * 255).astype(np.uint8))

            cell.axis("off")

        if row % 2 == 0:
            r_real += 1
        else:
            r_fake += 1

    plt.axis("off")
    
    fig.savefig(os.path.join(tgt_pth, '{}.jpg'.format(str(e).zfill(3))))
    
    plt.close()

## Load Batch

In [11]:
def load_noise():
    
    y_true = tf.ones((BATCH_SIZE,))
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.40

    with tf.Session(config=config) as sess:
        tf.initialize_all_variables().run()
        
        # run once
        y_true = y_true.eval()

        while True:
            values = sess.run([variable_dataset])
            row = values[0]

            sz = row['img'].shape[0]

            if sz != BATCH_SIZE:
                continue
            
            # fake data
            # concatenate face + age + noise
            c1 = row['enc']
            c2 = tf.cast(row['age'], tf.float32).eval()
            x = tf.random.normal((sz, noise_len,)).eval()
            
            yield c1, c2, x, y_true


def load_batch():
    
    y_fake = tf.zeros((BATCH_SIZE,))
    y_true = tf.ones((BATCH_SIZE,))
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.40

    with tf.Session(config=config) as sess:
        tf.initialize_all_variables().run()
        
        # run once
        y_fake = y_fake.eval()
        y_true = y_true.eval()

        while True:
            values = sess.run([variable_dataset])
            row = values[0]

            sz = row['img'].shape[0]

            if sz != BATCH_SIZE:
                continue
            
            # fake data
            c1 = row['enc']
            c2 = tf.cast(row['age'], tf.float32).eval()
            x = tf.random.normal((sz, noise_len,)).eval()
            z_fake = generator.predict([c1, c2, x])

            # real data
            c1 = row['enc']
            c2 = tf.cast(row['age'], tf.float32).eval()
            z_real = tf.reshape(tf.io.decode_raw(row['img'], tf.uint8), (-1, width, height, depth)).eval()
                        
            yield c1, c2, x, z_fake, y_fake, z_real, y_true

## Train Model

In [12]:
EPOCHS = 4000
STEPS = 1  # 60000 // BATCH_SIZE


train_loss_g = []
train_loss_d = []

train_acc_g = []
train_acc_d = []


disc_itr = load_batch()
gen_itr = load_noise()


# epochs
for e in range(EPOCHS):

    #batches
    loss = []
    acc = []

    for p in range(STEPS * 4):
        
        c1, c2, x, z_fake, y_fake, z_real, y_real = next(disc_itr)
    
        # train
        loss_2, acc_2, _, _, _ = discriminator.train_on_batch([c1, c2, z_real], [y_real, c2])
        loss_1, acc_1, _, _, _ = discriminator.train_on_batch([c1, c2, z_fake], [y_fake, c2])

        batch_loss = 0.5 * (loss_1 + loss_2)
        batch_acc = 0.5 * (acc_1 + acc_2)

        loss.append(batch_loss)
        acc.append(batch_acc)

    train_loss_d.append(np.mean(np.array(loss)))
    train_acc_d.append(np.mean(np.array(acc)))

    #batches
    loss = []
    acc = []

    for p in range(STEPS):

        c1, c2, x, y_true = next(gen_itr)

        # train
        loss_1, acc_1, _, _, _ = gan.train_on_batch([c1, c2, x], [y_true, c2])

        loss.append(loss_1)
        acc.append(acc_1)

    train_loss_g.append(np.mean(np.array(loss)))
    train_acc_g.append(np.mean(np.array(acc)))


    print("Epoch: {}, Steps: {}, Discriminator Accuracy: %{:.2f}, GAN Accuracy: %{:.2f}".format(
          e,
          STEPS,
          train_acc_d[-1] * 100,
          train_acc_g[-1] * 100
      ))

    if e % 10 == 0:
        ## visualize results
        c1, c2, x, z_fake, y_fake, z_real, y_real = next(disc_itr)
        visualizeGAN(e, z_real, z_fake)
        
        ## save model
        pth = os.path.join(models_path, 'gan.h5')
        gan.save(pth)

        pth = os.path.join(models_path, 'generator-{}-{}-{}.h5'.format(e, train_loss_g[-1], train_acc_g[-1]))
        generator.save(pth)

        pth = os.path.join(models_path, 'discriminator.h5')
        discriminator.save(pth)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch: 0, Steps: 1, Discriminator Accuracy: %13.97, GAN Accuracy: %48.27
Epoch: 1, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %48.53
Epoch: 2, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %48.43
Epoch: 3, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %46.71
Epoch: 4, Steps: 1, Discriminator Accuracy: %0.05, GAN Accuracy: %30.89
Epoch: 5, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %23.13
Epoch: 6, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %29.63
Epoch: 7, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %30.99
Epoch: 8, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %30.68
Epoch: 9, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %28.61
Epoch: 10, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %28.65
Epoch: 11, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %27.62
Epoch: 12, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %26.38
Epoch: 13, Steps: 1, Discriminator Accuracy: %0.01, GAN Accu

Epoch: 114, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %2.06
Epoch: 115, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %1.90
Epoch: 116, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %1.88
Epoch: 117, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %1.91
Epoch: 118, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %1.86
Epoch: 119, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %1.81
Epoch: 120, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %1.62
Epoch: 121, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %1.75
Epoch: 122, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %1.62
Epoch: 123, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %1.59
Epoch: 124, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %1.64
Epoch: 125, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %1.57
Epoch: 126, Steps: 1, Discriminator Accuracy: %0.00, GAN Accuracy: %1.58
Epoch: 127, Steps: 1, Discriminator Accuracy: %0.00

AssertionError: Nesting violated for default stack of <class 'tensorflow.python.client.session.Session'> objects

## Plot Loss

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_loss_g, label="Generator Loss");
plt.plot(train_loss_d, label="Discriminator Loss");
plt.legend();

## Plot Accuracy

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_acc_g, label="Generator Accuracy");
plt.plot(train_acc_d, label="Discriminator Accuracy");
plt.legend();